# Gender Changing Levels

In [ ]:
import pandas as pd
import pandas_ta as ta
df = pd.read_csv("data/monthly_segments/CL_30m_2024-09.csv")

df.columns=['datetime', 'open', 'high', 'low', 'close', 'volume']
#Check if NA values are in data
df=df[df['volume']!=0]
df.reset_index(drop=True, inplace=True)
df['atr14'] = ta.atr(df['high'], df['low'], df['close'], length=14)
df.head(10)

,datetime,open,high,low,close,volume,atr14
0,2024-09-01 18:00:00,73.53,73.65,73.39,73.41,214,NaN
1,2024-09-01 18:30:00,73.41,73.48,73.17,73.24,1485,NaN
2,2024-09-01 19:00:00,73.24,73.28,73.15,73.19,507,NaN
3,2024-09-01 19:30:00,73.20,73.22,73.09,73.12,471,NaN
4,2024-09-01 20:00:00,73.12,73.13,72.95,72.99,756,NaN
5,2024-09-01 20:30:00,72.97,73.18,72.97,73.17,749,NaN
6,2024-09-01 21:00:00,73.17,73.33,73.11,73.33,1360,NaN
7,2024-09-01 21:30:00,73.32,73.38,73.13,73.16,1343,NaN
8,2024-09-01 22:00:00,73.15,73.22,73.00,73.06,996,NaN
9,2024-09-01 22:30:00,73.06,73.06,72.90,72.94,1176,NaN


In [2]:
right_peek = 10
left_peek = 10

def pivotid(df1, index, left_peek, right_peek): #left_peek right_peek before and after candle at index. 
    if index-left_peek < 0 or index+right_peek >= len(df1): # Avoid out of bounds. 
        return 0
    
    pividlow=1
    pividhigh=1
    for i in range(index-left_peek, index+right_peek+1):
        if(df1.low[index]>df1.low[i]):
            pividlow=0 # Check if current index is lowest inside of window. 
        if(df1.high[index]<df1.high[i]):
            pividhigh=0 # Check if current index is highest inside of window.

    if pividlow and pividhigh: # Edge case: Both swing high and low. 
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0

# x.name is the numerical index, since we reset_index before. 
df[f'pivot'] = df.apply(lambda x: pivotid(df, x.name, right_peek, left_peek), axis=1)





## Pivot Candles Vizualisation

In [3]:
df.head(10)

,datetime,open,high,low,close,volume,atr14,pivot
0,2024-09-01 18:00:00,73.53,73.65,73.39,73.41,214,NaN,0
1,2024-09-01 18:30:00,73.41,73.48,73.17,73.24,1485,NaN,0
2,2024-09-01 19:00:00,73.24,73.28,73.15,73.19,507,NaN,0
3,2024-09-01 19:30:00,73.20,73.22,73.09,73.12,471,NaN,0
4,2024-09-01 20:00:00,73.12,73.13,72.95,72.99,756,NaN,0
5,2024-09-01 20:30:00,72.97,73.18,72.97,73.17,749,NaN,0
6,2024-09-01 21:00:00,73.17,73.33,73.11,73.33,1360,NaN,0
7,2024-09-01 21:30:00,73.32,73.38,73.13,73.16,1343,NaN,0
8,2024-09-01 22:00:00,73.15,73.22,73.00,73.06,996,NaN,0
9,2024-09-01 22:30:00,73.06,73.06,72.90,72.94,1176,NaN,0


In [4]:
import numpy as np

def pointpos(x, distance):
    if x[f'pivot']==1:
        return x['low']-distance
    elif x[f'pivot']==2:
        return x['high']+distance
    else:
        return np.nan


# Point position for help plotting. 
df['pointpos'] = df.apply(lambda row: pointpos(row, distance=0.2), axis=1)

In [5]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'],
                increasing_line_color= 'green', 
                decreasing_line_color= 'red')])

fig.add_scatter(x=df.index, y=df['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.update_layout(paper_bgcolor='black', plot_bgcolor='black')

fig.show()

In [6]:
import numpy as np

# Count how often pivot highs/lows land in the same price zone (>=2 hits become bands)
bin_width = 0.10  # 10 pips-wide zones
MIN_TOUCHES = 4

support_slice = df.loc[df['pivot'] == 1, ['low', 'datetime']].dropna()
support_prices = support_slice['low']
support_dates = support_slice['datetime']

resistance_slice = df.loc[df['pivot'] == 2, ['high', 'datetime']].dropna()
resistance_prices = resistance_slice['high']
resistance_dates = resistance_slice['datetime']

support_levels = (support_prices / bin_width).round() * bin_width
resistance_levels = (resistance_prices / bin_width).round() * bin_width

# Would need to add a created at. 
level_summary = (
    pd.concat([
        pd.DataFrame({'level': support_levels, 'support_hits': 1, 'resistance_hits': 0, 'created': support_dates, 'type': 'support'}),
        pd.DataFrame({'level': resistance_levels, 'support_hits': 0, 'resistance_hits': 1, 'created': resistance_dates, 'type': 'resistance'})
    ])
    .groupby('level', as_index=False)
    .agg({'support_hits':'sum','resistance_hits':'sum','created':'min', 'type':'first'})
)
level_summary['total_hits'] = level_summary['support_hits'] + level_summary['resistance_hits']
level_summary = level_summary[level_summary['total_hits'] >= MIN_TOUCHES].sort_values('total_hits', ascending=False)
level_summary.head()


,level,support_hits,resistance_hits,created,type,total_hits
29,69.2,8,3,2024-09-04 04:00:00,support,11
25,68.8,5,4,2024-09-04 16:00:00,support,9
30,69.3,5,2,2024-09-05 06:30:00,support,7
31,69.4,2,5,2024-10-17 13:30:00,support,7
38,70.1,5,2,2024-09-03 14:30:00,support,7


In [7]:
import plotly.graph_objects as go

# Highlight zones where we have at least two combined hits
zones = level_summary[level_summary['total_hits'] >= 2].copy()
zones['created'] = pd.to_datetime(zones['created'])

if zones.empty:
    print('No multi-hit zones found')
else:
    df_band = df.copy()
    df_band['datetime'] = pd.to_datetime(df_band['datetime'])

    fig = go.Figure(data=[go.Candlestick(
        x=df_band['datetime'],
        open=df_band['open'],
        high=df_band['high'],
        low=df_band['low'],
        close=df_band['close'],
        increasing_line_color='green',
        decreasing_line_color='red'
    )])

    half_width = bin_width / 2
    for _, zone in zones.iterrows():
        is_support = zone['support_hits'] > zone['resistance_hits']
        fill = 'rgba(72, 209, 204, 0.25)' if is_support else 'rgba(255, 0, 0, 0.25)'
        outline = 'rgba(72, 209, 204, 0.8)' if is_support else 'rgba(255, 0, 0, 0.8)'
        fig.add_shape(
            type='rect',
            xref='x',
            yref='y',
            x0=zone['created'],
            x1=df_band['datetime'].iloc[-1],
            y0=zone['level'] - half_width,
            y1=zone['level'] + half_width,
            line=dict(color=outline, width=1),
            fillcolor=fill
        )

    fig.update_layout(
        xaxis_rangeslider_visible=False,
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False),
        paper_bgcolor='black',
        plot_bgcolor='black',
        title='Support/Resistance Bands (>=2 hits)'
    )

    fig.show()


In [8]:
level_segments = []
flip_markers = []
threshold_mult = 2.0

df_levels = df.copy()
df_levels['datetime'] = pd.to_datetime(df_levels['datetime'])
zones_eval = zones.copy()
zones_eval['created'] = pd.to_datetime(zones_eval['created'])
end_time = df_levels['datetime'].iloc[-1]

for _, zone in zones_eval.iterrows():
    current_type = zone['type']
    level_price = zone['level']
    current_start = zone['created']
    window = df_levels[df_levels['datetime'] >= current_start]

    if window.empty:
        continue

    for _, candle in window.iterrows():
        atr_value = candle['atr14']
        if pd.isna(atr_value):
            continue
        threshold = atr_value * threshold_mult

        if current_type == 'resistance':
            if candle['high'] >= level_price + threshold:
                if candle['datetime'] > current_start:
                    level_segments.append({
                        'level': level_price,
                        'start': current_start,
                        'end': candle['datetime'],
                        'type': current_type
                    })
                flip_markers.append({'level': level_price, 'time': candle['datetime'], 'new_type': 'support'})
                current_type = 'support'
                current_start = candle['datetime']
        else:
            if candle['low'] <= level_price - threshold:
                if candle['datetime'] > current_start:
                    level_segments.append({
                        'level': level_price,
                        'start': current_start,
                        'end': candle['datetime'],
                        'type': current_type
                    })
                flip_markers.append({'level': level_price, 'time': candle['datetime'], 'new_type': 'resistance'})
                current_type = 'resistance'
                current_start = candle['datetime']

    if end_time > current_start:
        level_segments.append({
            'level': level_price,
            'start': current_start,
            'end': end_time,
            'type': current_type
        })

level_segments_df = pd.DataFrame(level_segments) if level_segments else pd.DataFrame(columns=['level','start','end','type'])
flip_markers_df = pd.DataFrame(flip_markers) if flip_markers else pd.DataFrame(columns=['level','time','new_type'])
level_segments_df.head()


,level,start,end,type
0,69.2,2024-09-04 04:00:00,2024-09-06 11:00:00,support
1,69.2,2024-09-06 11:00:00,2024-09-13 05:00:00,resistance
2,69.2,2024-09-13 05:00:00,2024-09-13 14:30:00,support
3,69.2,2024-09-13 14:30:00,2024-09-16 09:30:00,resistance
4,69.2,2024-09-16 09:30:00,2024-09-18 06:00:00,support


In [9]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(
    x=df_levels['datetime'],
    open=df_levels['open'],
    high=df_levels['high'],
    low=df_levels['low'],
    close=df_levels['close'],
    increasing_line_color='green',
    decreasing_line_color='red'
)])

legend_shown = {'support': False, 'resistance': False}

if not level_segments:
    print('No level segments to display.')
else:
    for segment in level_segments:
        color = 'MediumSeaGreen' if segment['type'] == 'support' else 'Tomato'
        fig.add_trace(go.Scatter(
            x=[segment['start'], segment['end']],
            y=[segment['level'], segment['level']],
            mode='lines',
            line=dict(color=color, width=2),
            name=f"{segment['type'].title()} level",
            legendgroup=segment['type'],
            showlegend=not legend_shown[segment['type']]
        ))
        legend_shown[segment['type']] = True

if flip_markers:
    fig.add_trace(go.Scatter(
        x=[marker['time'] for marker in flip_markers],
        y=[marker['level'] for marker in flip_markers],
        mode='markers',
        marker=dict(color='yellow', size=9, symbol='diamond'),
        name='Gender change'
    ))

fig.update_layout(
    xaxis_rangeslider_visible=False,
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    paper_bgcolor='black',
    plot_bgcolor='black',
    title='Levels with Gender Changes'
)

fig.show()
